In [1]:
import pandas as pd

df = pd.read_parquet("../parquets/clean_detection_results_debug_images.parquet")

In [2]:
df

,filename,score,col_x,row_y,width,height,label,outlier_distance,outlier
0,../debug_images/0000862a233105cc.jpg,0.832481,316.5,447.0,155,118,person,NaN,False
1,../debug_images/002e555bd63acb71.jpg,0.853143,696.0,347.0,648,670,person,NaN,False
2,../debug_images/002e555bd63acb71.jpg,0.791996,166.5,624.0,141,112,person,NaN,False
3,../debug_images/002e555bd63acb71.jpg,0.759568,66.5,599.5,129,163,person,NaN,False
4,../debug_images/002e555bd63acb71.jpg,0.757808,289.0,590.5,122,183,person,NaN,False
...,...,...,...,...,...,...,...,...,...
1053,../debug_images/138c4698f334e5ff.jpg,0.380026,529.5,396.0,989,570,person,NaN,False
1054,../debug_images/13a887093768b5ec.jpg,0.710493,495.5,440.5,245,209,person,NaN,False
1055,../debug_images/13c67cdd79ab35f0.jpg,0.407185,450.5,271.5,275,157,person,NaN,False
1056,../debug_images/13c67cdd79ab35f0.jpg,0.350427,601.5,354.5,279,149,person,NaN,False


In [3]:
df.dropna(inplace=True)

In [4]:
def yolobbox2bbox(x,y,w,h):
    x1, y1 = x-w/2, y-h/2
    x2, y2 = x+w/2, y+h/2
    return int(x1), int(y1), int(x2), int(y2)

In [5]:
from pascal_voc_writer import Writer
import cv2
from pathlib import Path
from tqdm.auto import tqdm


for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Converting annotations into XML"):  
    filename = row['filename']
    label = row['label']
    cx, cy, w, h = row['col_x'], row['row_y'], row['width'], row['height']
    
    x1, y1, x2, y2 = yolobbox2bbox(cx, cy, w, h)
    
    image = cv2.imread(filename)
    height, width = image.shape[:2]


    writer = Writer(filename, width, height)
    writer.addObject(label, x1, y1, x2, y2)

    filename_path = Path(filename)
    xml_path = str(filename_path.with_suffix(".xml"))
    writer.save(xml_path)


/home/dnth/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Converting annotations into XML: 100%|██████████| 49/49 [00:00<00:00, 88.26it/s]
